# **Análisis exploratorio de datos** (`EDA`)

### Importamos librerías y cargamos los archivos de entrada

In [33]:
#importamos librerias necesarias
import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point
import warnings
warnings.filterwarnings("ignore")

In [34]:
#Leemos las bases de datos
air_quality = pd.read_parquet(r'C:\Users\melan\Desktop\PG 1\ETL -EDA\Data\2. Normalizacion\air_quality.parquet')
sound_quality = pd.read_parquet(r'C:\Users\melan\Desktop\PG 1\ETL -EDA\Data\2. Normalizacion\sound_quality.parquet')
climate = pd.read_parquet(r'C:\Users\melan\Desktop\PG 1\ETL -EDA\Data\2. Normalizacion\climate_NY.parquet')


## EDA `Air Quality`

In [35]:
air_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pollutant     10 non-null     object 
 1   Borough       10 non-null     object 
 2   BoroughID     10 non-null     float64
 3   Measure Info  10 non-null     object 
 4   Year          10 non-null     int64  
 5   Data Value    10 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 612.0+ bytes


In [68]:
import plotly.express as px

# Creamos el gráfico de barras horizontales
fig = px.bar(air_quality, y='Borough', x='Data Value', color='Pollutant',
             labels={'Data Value': 'Niveles de Contaminantes (mcg/m3 or ppb)'},
             title='Niveles de Contaminantes por Borough (2023)',
             template='plotly_white')

# Añadimos las etiquetas del total general a la derecha de cada barra
for i, value in enumerate(total_values['Data Value']):
    fig.add_annotation(
        x=value + 0.5,
        y=total_values['Borough'][i],
        text=f'Total: {value}',
        showarrow=True,
        arrowhead=5,
        ax=0,
        ay=-40
    )

# Añadimos bordes a las barras
fig.update_traces(marker_line=dict(color='black', width=1.5))

# Ajustamos el diseño del gráfico
fig.update_layout(
    legend_title='Contaminante',
    yaxis_title='Borough',
    xaxis_title='Niveles de Contaminantes (mcg/m3 or ppb)',
)

# Mostramos el gráfico
fig.show()


#### `Observaciones:` El Distrito de Manhattan es el más contaminado de la ciudad de Nueva York.

## EDA`Sound Quality`

In [37]:
sound_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762 entries, 0 to 2761
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Borough       2762 non-null   int64  
 1   Latitude      2762 non-null   float64
 2   Longitude     2762 non-null   float64
 3   Year          2762 non-null   int64  
 4   Day           2762 non-null   int64  
 5   Hour          2762 non-null   int64  
 6   Engine Sound  2762 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 151.2+ KB


In [38]:
# Realizamos el conteo de frecuencia
counts = sound_quality['Engine Sound'].value_counts()

# Definimos el orden de las categorías
categories_order = ['Low', 'Medium', 'High']

# Reordenamos los resultados de acuerdo al orden definido
counts = counts.reindex(categories_order)

# Creamos el gráfico de barras con Plotly
fig = px.bar(x=counts.index, y=counts.values, color=counts.index,
             labels={'x': 'Nivel de Sonido', 'y': 'Frecuencia'},
             title='Distribución de Niveles de Sonido',
             template='plotly_white')

# Añadimos etiquetas de conteo encima de las barras con fuente en negrita y flecha
for i, count in enumerate(counts):
    fig.add_annotation(
        x=counts.index[i],
        y=count + 1,
        text=str(count),
        showarrow=True,
        arrowhead=5,
        ax=0,
        ay=-40,
        font=dict(size=12, color='black', family='Arial')  # Configuramos la fuente en negrita
    )

# Añadimos bordes a las columnas
fig.update_traces(marker_line=dict(color='black', width=1.5))

# Ajustamos el diseño del gráfico
fig.update_layout(
    xaxis_title='Nivel de Sonido',
    yaxis_title='Frecuencia',
)

# Mostramos el gráfico
fig.show()

#### `Observaciones:` Se registra un conteo superior de ruidos medios en la Ciudad de New York.

#### El archivo de extensión .geojson utilizado para el GeoDataFrame se obtuvo de este [enlace](https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON)

In [39]:
# Cargamos el GeoJSON
with open(r'C:\Users\melan\Desktop\PG 1\ETL -EDA\Data\3. Extras\NYC Taxi Zones.geojson') as fd:
    geojson = json.load(fd)

# Creamos el mapa con Plotly
fig = go.Figure(go.Scattermapbox(
    mode='markers',
    lon=sound_quality['Longitude'],
    lat=sound_quality['Latitude'],
    marker=dict(
        size=10,
        color='red',
    ),
    text=sound_quality['Engine Sound'],
))

# Configuramos diseño del mapa
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=10,
        center=dict(lat=sound_quality['Latitude'].mean(), lon=sound_quality['Longitude'].mean()),
    ),
    height=600,
    title='Mapa de Sonidos de Motores en Nueva York',
    paper_bgcolor='#2c3e50',  # Color de fondo elegante (azul oscuro)
    font_color='white',  # Color del texto en blanco
)

# Mostramos el mapa
fig.show()


#### `Observaciones:` La mayor concentración de ruidos de motor en New York se encuentra en el Distrito de Manhattan.

In [64]:
import plotly.express as px
import plotly.graph_objects as go

# Creamos el DataFrame para contar la frecuencia por hora
hour_counts = sound_quality['Hour'].value_counts().sort_index().reset_index()
hour_counts.columns = ['Hour', 'Frequency']

# Crear el gráfico de área
fig = px.area(hour_counts, x='Hour', y='Frequency',
              labels={'Frequency': 'Frecuencia', 'Hour': 'Hora del Día'},
              title='Frecuencia de Sonidos Registrados por Hora del Día',
              template='plotly_white')

# Añadir marcadores en los puntos máximos y mínimos
max_frequency_hour = hour_counts.loc[hour_counts['Frequency'].idxmax(), 'Hour']
min_frequency_hour = hour_counts.loc[hour_counts['Frequency'].idxmin(), 'Hour']
fig.add_trace(go.Scatter(x=[max_frequency_hour, min_frequency_hour],
                         y=[hour_counts['Frequency'].max(), hour_counts['Frequency'].min()],
                         mode='markers',
                         marker=dict(color='red', size=10),
                         showlegend=False))

# Ajustar el diseño del gráfico
fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=list(range(24)), ticktext=[f"{i:02}:00" for i in range(24)]),
    yaxis_title='Frecuencia',
)

# Mostrar el gráfico
fig.show()



#### `Observaciones:` De 08:00 a 20:00 Hs es el lapso donde mas ruidos de motor se registran en New York.

## EDA `Climate`

In [41]:
climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6552 entries, 0 to 6551
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Temperature (°C)  6552 non-null   object
 1   Date              6552 non-null   object
 2   Time              6552 non-null   object
dtypes: object(3)
memory usage: 153.7+ KB


In [42]:
# Convertimos la columna de temperatura a tipo numérico
climate['Temperature (°C)'] = pd.to_numeric(climate['Temperature (°C)'], errors='coerce')

In [53]:
import plotly.graph_objects as go

# Creamos el gráfico de área con Plotly
fig = go.Figure()

# Área de temperatura
fig.add_trace(go.Scatter(x=climate['Date'], y=climate['Temperature (°C)'],
                         mode='lines',
                         fill='tozeroy',  # Relleno hasta el eje y=0
                         fillcolor='rgba(0, 0, 255, 0.3)',  # Color del área
                         line=dict(color='blue'),  # Color de la línea
                         name='Temperatura'))

# Identificamos los puntos más altos y más bajos
max_temperature_date = climate.loc[climate['Temperature (°C)'].idxmax(), 'Date']
max_temperature_value = climate['Temperature (°C)'].max()
min_temperature_date = climate.loc[climate['Temperature (°C)'].idxmin(), 'Date']
min_temperature_value = climate['Temperature (°C)'].min()

# Añadimos marcadores para los puntos de temperatura máxima y mínima
fig.add_trace(go.Scatter(x=[max_temperature_date], y=[max_temperature_value],
                         mode='markers',
                         marker=dict(color='red', size=10),
                         name='Temperatura máxima'))
fig.add_trace(go.Scatter(x=[min_temperature_date], y=[min_temperature_value],
                         mode='markers',
                         marker=dict(color='green', size=10),
                         name='Temperatura mínima'))

# Configuramos el diseño del gráfico
fig.update_layout(
    title='Variación de la Temperatura a lo largo del Tiempo',
    xaxis_title='Fecha',
    yaxis_title='Temperatura (°C)',
    template='plotly_white'
)

# Mostramos el gráfico
fig.show()



#### `Observaciones:` La temperatura promedio en el año 2023 fue de 14.26ºC. Durante los primeros cuatro meses del año, de Enero a Abril, la temperatura estuvo por debajo de esa media. Sin embargo, a partir de mayo, experimentó un aumento gradual hasta alcanzar su punto máximo en Septiembre. A partir de ese mes, la temperatura comenzó a descender nuevamente. Este patrón sugiere una variación estacional significativa a lo largo del año, con una tendencia a temperaturas más bajas en los primeros meses, seguida de un aumento progresivo y luego un descenso hacia finales del año.

In [49]:
import plotly.express as px

# Creamos el gráfico de barras con Plotly
fig = px.bar(climate, x='Time', y='Temperature (°C)',
             labels={'Temperature (°C)': 'Temperatura (°C)', 'Time': 'Hora'},
             title='Distribución de Temperaturas por Hora',
             template='plotly_white',
             barmode='group')

# Ajustamos el diseño del gráfico
fig.update_layout(
    xaxis_title='Hora',
    yaxis_title='Temperatura (°C)',
)

# Mostramos el gráfico
fig.show()


#### `Observaciones:` La temperatura asciende a partir de las 08:00 para descender a partir de las 16:00. Se aprecian valores atípicos entre las 03:00 y las 06:00 que se corresponden con las bajas temperaturas registradas en fecha 04/02/2023.

In [57]:
# Creamos el gráfico de histograma con etiquetas y bordes
fig = px.histogram(climate, x='Temperature (°C)', nbins=30,
                   labels={'Temperature (°C)': 'Temperatura (°C)', 'count': 'Frecuencia'},
                   title='Distribución de Temperaturas',
                   template='plotly_white')

# Añadimos etiquetas a las columnas
fig.add_trace(go.Scatter(x=[climate['Temperature (°C)'].mean()]*2,
                         y=[0, fig.data[0].y],
                         text=[f'Media: {climate["Temperature (°C)"].mean():.2f}°C']*2,
                         mode='text',
                         showlegend=False))

# Marcamos los bordes de las columnas
fig.update_traces(marker_line=dict(color='black', width=1))

# Ajustamos el diseño del gráfico
fig.update_layout(
    xaxis_title='Temperatura (°C)',
    yaxis_title='Frecuencia',
)

# Mostramos el gráfico
fig.show()

#### `Observaciones:` Existen mas registros en 20° C.